# Solute Transport and the principle of superposition, an example

We have an aquifer with a saturated thickness of 20 m, porosity of 0.1, and hydraulic conductivity of 5 m/d. The river is completely contaminated by two pollution episodes separated by 5 days. The first pollution episode can be represented as a constant pulse with a concentration of 800 mg/L that lasts for 30 days. The second episode is a constant pulse with a concentration of 600 mg/L that lasts for 40 days. After 80 days from the start of the contamination, the river returns to its initial chemical state (zero chlorides). It is assumed that chlorides are neither adsorbed nor degraded in the aquifer. 
<br>
<br>
![alt text](figure-1.PNG)
<br>
<br>
The following is requested:\
**(A)** Calculate the travel time of the chlorides to point A.\
**(B)** Calculate the concentration of chlorides at point A after 440 days from the start of the river contamination.\
**(C)** Calculate the maximum concentration of chlorides that will reach point A.


#### Importing the needed libraries

In [1]:
#-- Check and install required packages if not already installed
import sys
import subprocess

def if_require(package):
    try:
        __import__(package)
    except ImportError:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        
#-- Install packages only if they aren't already installed 
if_require("ipywidgets")

#-- Import the rest of libraries 
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy import special
import warnings
warnings.filterwarnings("ignore")

#### Coding the equations as functions

In [2]:
def advective_velocity(K, i, phi):
    """Calculates the advective velocity.

    Parameters
    ----------
    K : float
        hydraulic conductivity
    i : float
        hydraulic gradient, as the change in water level per unit of distance.
    phi : float
        porosity

    Returns
    -------
    float
        advective velocity
    """
    v = ((K*i))/phi
    return v
   
def travel_time(k1, i1, phi1, x1, k2, i2, phi2, x2):
    """Calculates the travel time with the velocities from the two Sectors.

    Parameters
    ----------
    k1 : float
        Hydraulic Conductivity from the first Sector
    i1 : float
        Hydraulic Gradient from the first Sector
    phi1 : float
        Porosity form the first Sector
    x1 : float
        Distance from the River to the second Sector
    k2 : float
        Hydraulic Conductivity from the second Sector
    i2 : float
        Hydraulic Gradient from the first Sector
    phi2 : float
        Porosity form the second Sector
    x2 : float
        Distance from the second Sector to the Point A

    Returns
    -------
    float(s)
        velocity in the first Sector; velocity in the second Sector; the travel time
    """
    
    v1 = advective_velocity(k1, i1, phi1)
    v2 = advective_velocity(k2, i2, phi2)    
    
    t = (x1/v1) + (x2/v2)
    
    return v1, v2, t   
    
def dispersivity(x):
    """Estimates dispersivity with a semi-empirical equation.

    Parameters
    ----------
    x : floar
        total distance from the river to Point A

    Returns
    -------
    float
        dispersivity
    """
    alpha = 0.83*(np.log10(x))**2.414
    return alpha
   
def dispersion(alpha, v):
    """Calculates the dispersion.

    Parameters
    ----------
    alpha : flaot
        dispersivity 
    v : float
        velocity

    Returns
    -------
    float
        dispersion
    """
    D = alpha * v
    return D

def oneD_continous_injection(Ci, x, v, t, D):
    """Function that calculates the concentration in a one-dimensional continous injection,
    with no adsorption or decay

    Parameters
    ----------
    Ci : float
        initial concentration
    x : float
        x position
    v : float
        velocity
    t : float
        time
    D : float
        dispersion

    Returns
    -------
    float
        concentration 
    """
    
    term1 = special.erfc((x - (v*t))/(np.sqrt(4*D*t)))
    term2 = np.exp((v*x)/D)
    term3 = special.erfc(x+(v*t)/np.sqrt(4*D*t))
       
    if (v*x)/D > 100:
        C = (Ci/2) * (term1)
    else:
        C = (Ci/2) * (term1 + term2 * term3)
    
    return C


#### Coding the widgets

In [ ]:
def create_widget(description: str, style=None, width=None):
    """
    Function for the generation of widget 

    Parameters:
        - description (str): description of the widget
    Return:
        - widget_value: returns the value of the widget
    """ 
    widget_value = widgets.Text(value='', description=description, 
                                style={'description_width': 'initial'}, width = '500px')
    return widget_value


def dropdown_widget(description: str, options: str, value: str):
    """
    Function for the generation of the dropdown widget.
    
    Parameters:
        - description (str): description of the widget
        - options (str): the values to choose in the widget
        - value (str): the value to initialize the widget
    Return:
        - widget_value: returns the value of the widget
    """ 
    widget_value = widgets.Dropdown(description=description, options=options, 
                                    value = value, style = {'description_width': 'initial'},
                                    width = '400px', disabled=False)
    return widget_value
 
def update_numeric_value(widget, variable):
    """Observes changes in the widget's value and updates the variable accordingly.

    When a user enters or modifies text in the widget, the 'update_value' function 
    will be automatically called, allowing you to respond to and handle the changes 
    in the widget's value. 

    Parameters
    ----------
    widget : ipywidgets.Widget
        The widget object that will be observed for changes in its value
    variable : Any
        A global variable to be updated based on the widget's input

    Returns
    -------
    widget : ipywidgets.Widget
        The original widget, with an observer attached to monitor changes

    Notes
    -----
    If the new value in the widget is blank, 'variable' will be set to None. If the 
    input is invalid, an error message will be displayed
    """
    def update_value(change):
        global variable
        new_value = change.new.strip()
        if new_value == '':
            variable = None
        else:
            try:
                variable = float(new_value)
            except ValueError:
                print(f"Invalid input for '{widget.description}', please enter a valid value.")

    widget.observe(update_value, names='value')
    return widget


def k1_input(k1):
    """
    Hydraulic conductivity Sector 1
    """
    k1_wid = create_widget(description='Hydraulic conductivity Sector 1 [m/d]:',
                           style={'description_width': 'initial'}, width='500px')  
    k1_wid = update_numeric_value(k1_wid, k1)
    return k1_wid

def k2_input(k2):
    """
    Hydraulic conductivity Sector 2
    """
    k2_wid = create_widget(description='Hydraulic conductivity Sector 2 [m/d]:',
                           style={'description_width': 'initial'}, width='500px')  
    k2_wid = update_numeric_value(k2_wid, k2)
    return k2_wid

def i1_input(i1):
    """
    Gradient Sector 1
    """
    i1_wid = create_widget(description='Gradient Sector 1 [-]:',
                           style={'description_width': 'initial'}, width='500px')  
    i1_wid = update_numeric_value(i1_wid, i1)
    return i1_wid

def i2_input(i2):
    """
    Gradient Sector 2
    """
    i2_wid = create_widget(description='Gradient Sector 2 [-]:',
                           style={'description_width': 'initial'}, width='500px')  
    i2_wid = update_numeric_value(i2_wid, i2)
    return i2_wid

def phi1_input(phi1):
    """
    Porosity Sector 1
    """
    phi1_wid = create_widget(description='Porosity Sector 1 [-]:',
                           style={'description_width': 'initial'}, width='500px')  
    phi1_wid = update_numeric_value(phi1_wid, phi1)
    return phi1_wid

def phi2_input(phi2):
    """
    Porosity Sector 2
    """
    phi2_wid = create_widget(description='Porosity Sector 2 [-]:',
                           style={'description_width': 'initial'}, width='500px')  
    phi2_wid = update_numeric_value(phi2_wid, phi2)
    return phi2_wid

def x1_input(x1):
    """
    Distance Sector 1
    """
    x1_wid = create_widget(description='Distance to Sector 2 [m]:',
                           style={'description_width': 'initial'}, width='500px')  
    x1_wid = update_numeric_value(x1_wid, x1)
    return x1_wid

def x2_input(x2):
    """
    Distance Sector 2
    """
    x2_wid = create_widget(description='Distance to Point A [m]:',
                           style={'description_width': 'initial'}, width='500px')  
    x2_wid = update_numeric_value(x2_wid, x2)
    return x2_wid


#-- Button to trigger the calculation
output = widgets.Output()
button = widgets.Button(button_style='info', description="Click to solve",
                            tooltip='Click me', icon='check', 
                            layout=widgets.Layout(width='300px', height='30px'))

C_values={}
def on_button_clicked(button):
    k1 = float(k1_wid.value) 
    k2 = float(k2_wid.value)  
    i1 = float(i1_wid.value) 
    i2 = float(i2_wid.value) 
    phi1 = float(phi1_wid.value) 
    phi2 = float(phi2_wid.value) 
    x1 = float(x1_wid.value) 
    x2 = float(x2_wid.value) 
    
    v1, v2, arrival_time = travel_time(k1, i1, phi1, x1, k2, i2, phi2, x2)
    total_distance = x1 + x2
    mean_velocity = total_distance/arrival_time
    alpha = dispersivity(total_distance)
    D = dispersion(alpha, mean_velocity)
    
    time = np.arange(1,1001,1)
    Ca = np.zeros(len(time))
    Cb = np.zeros(len(time))
    Cc = np.zeros(len(time))
    Cd = np.zeros(len(time))
    
    for t in range(len(Ca)):
        Ca[t] = oneD_continous_injection(800, total_distance, mean_velocity, time[t], D)      
    
    for t in range(len(Ca)):    
        if t>30:      
            Cb[t] = oneD_continous_injection(800, total_distance, mean_velocity, time[t]-30, D)
            
    for t in range(len(Ca)):   
        if t>35:        
            Cc[t] = oneD_continous_injection(600, total_distance, mean_velocity, time[t]-35, D)
    
    for t in range(len(Ca)):   
        if t>75:                  
            Cd[t] = oneD_continous_injection(600, total_distance, mean_velocity, time[t]-75, D)
    
    Ctot = Ca - Cb + Cc - Cd
    
    C_values["Ca"] = Ca
    C_values["Cb"] = Cb
    C_values["Cc"] = Cc
    C_values["Cd"] = Cd
    C_values["Ctot"] = Ctot
         
    with output:
        output.clear_output(wait=True)        
        Ctot_list = list(Ctot)
        idx_max = Ctot_list.index(np.nanmax(Ctot_list))
        print(f"A - The travel time to point A is {arrival_time:.2f} days\n")
        print(f"B - The concentration value in day 440 is {Ctot[440]:.2f} mg/l\n")
        print(f"C - The max value is {np.nanmax(Ctot_list):.2f} mg/l\n")
        
        
        plt.figure(figsize=(6, 5))
        plt.plot(time, Ctot)  
        plt.xlabel('time [d]')
        plt.ylabel('$C$ [mg/l]')
        plt.ylim(0, 1000)
        plt.xlim(0, 1000)
        plt.scatter(time[idx_max], np.nanmax(Ctot), color="red")
        plt.scatter(440, Ctot[440], color="blue")  
        plt.plot([440, 440], [0, Ctot[440]], color="blue", ls=":")
        plt.plot([time[idx_max], time[idx_max]], [0, Ctot[idx_max]], color="red", ls=":")
        plt.plot([0, 440], [Ctot[440], Ctot[440]], color="blue", ls=":")
        plt.plot([0, time[idx_max]], [Ctot[idx_max], Ctot[idx_max]], color="red", ls=":")
        plt.show()
 
#-- Attaching the function to the button's click event
button.on_click(on_button_clicked)

#-- Displaying the widgets and button
k1_wid = k1_input(k1=0)  
x1_wid = x1_input(x1=0)
i1_wid = i1_input(i1=0)
phi1_wid = phi1_input(phi1=0)

k2_wid = k2_input(k2=0)  
x2_wid = x2_input(x2=0)  
i2_wid = i2_input(i2=0)
phi2_wid = phi2_input(phi2=0)


#-- Defining the order of widgets
widgets_list = [k1_wid, x1_wid, i1_wid, phi1_wid, k2_wid, x2_wid, i2_wid, phi2_wid, button]

# Splitting the widgets into two columns
column1_widgets = widgets_list[:len(widgets_list) // 2]  
column2_widgets = widgets_list[len(widgets_list) // 2:]  

# Create vertical boxes for each column
column1_box = widgets.VBox(column1_widgets)
column2_box = widgets.VBox(column2_widgets)

# Arrange the columns side by side in a horizontal box
two_column_layout = widgets.HBox([column1_box, column2_box])

# Display the two-column layout
display(two_column_layout, output)

Output()